In [ ]:
import os
from PIL import Image

def check_images(directory):
    for file_name in os.listdir(directory):
        file_path = os.path.join(directory, file_name)
        try:
            with Image.open(file_path) as img:
                img.verify()  # Verify that the file is a valid image
        except (IOError, SyntaxError) as e:
            print(f"Corrupted file: {file_path}")

check_images(r"/content/extracted_files/dataset_new/msumsfd_gfacnn/attack")
check_images(r"/content/extracted_files/dataset_new/msumsfd_gfacnn/real")


In [3]:
# Install gdown if not already installed
!pip install gdown

# Download the zip file from Google Drive
import gdown
import zipfile

# Google Drive URL with file id for direct download
#url = 'https://drive.google.com/uc?id=1FwQ1lI2Btbzv2ksIVrcC1P2YSotyDeMd&export=download'
url = 'https://drive.google.com/uc?export=download&id=11kd0T-jJf72f4Jx6ioKRt1tWupxWaupg'
output = 'file.zip'

# Download the file
gdown.download(url, output, quiet=False)

# Unzip the file
with zipfile.ZipFile(output, 'r') as zip_ref:
    zip_ref.extractall('extracted_files')

print("File downloaded and unzipped successfully.")


Downloading...
From (original): https://drive.google.com/uc?export=download&id=11kd0T-jJf72f4Jx6ioKRt1tWupxWaupg
From (redirected): https://drive.google.com/uc?export=download&id=11kd0T-jJf72f4Jx6ioKRt1tWupxWaupg&confirm=t&uuid=b28f6509-3bea-4ad8-aa75-2f733c91647e
To: /content/file.zip
100%|██████████| 217M/217M [00:09<00:00, 22.8MB/s]


File downloaded and unzipped successfully.


In [5]:
import os
import random
import yaml
import tensorflow as tf
from sklearn import preprocessing
from sklearn.utils import shuffle
from tqdm import tqdm


class Dataset:
    def __init__(self, name, batch_size, attack_dir, real_dir):  # Fixed __init__
        self.name = name
        self.batch_size = batch_size
        self.attack_dir = attack_dir
        self.real_dir = real_dir
        self.dataset = MsuMsfdDataset(batch_size, attack_dir, real_dir)

        file_path, label_truth = self.dataset.load_idx()

        # Encode the labels
        self.encoding_truth = preprocessing.LabelEncoder()
        self.encoding_truth.fit(label_truth)
        self.list_label_truth = self.encoding_truth.transform(label_truth)

        self.list_file_path_truth = file_path.copy()

        self.shuffle_dataset()
        self.len_dataset = len(self.list_file_path_truth)


        self.standard_img_size = 224

        # Build dataset using tf.data for better performance
        self.train_dataset = self.build_tf_dataset()

    def shuffle_dataset(self):
        self.list_file_path_truth, self.list_label_truth = shuffle(self.list_file_path_truth, self.list_label_truth,
                                                                   random_state=10)
    def load_and_preprocess_image(self, file_path, label):
        img = tf.io.read_file(file_path)

        # Extract the file extension
        file_extension = tf.strings.substr(file_path, -4, 4)  # Get the last 4 characters

        # Check if the file is a JPEG or PNG
        is_jpg = tf.strings.regex_full_match(file_extension, '.jpg|.jpeg')
        is_png = tf.strings.regex_full_match(file_extension, '.png')

        # Define the image decoding function based on the file extension
        def decode_jpeg_image():
            img_decoded = tf.image.decode_jpeg(img, channels=3)
            img_resized = tf.image.resize(img_decoded, [self.standard_img_size, self.standard_img_size])
            return tf.image.convert_image_dtype(img_resized, dtype=tf.float32)

        def decode_png_image():
            img_decoded = tf.image.decode_png(img, channels=3)
            img_resized = tf.image.resize(img_decoded, [self.standard_img_size, self.standard_img_size])
            return tf.image.convert_image_dtype(img_resized, dtype=tf.float32)

        def skip_image():
            # Handle the case where the image format is not supported
            img_placeholder = tf.zeros([self.standard_img_size, self.standard_img_size, 3], dtype=tf.uint8)
            return tf.image.convert_image_dtype(img_placeholder, dtype=tf.float32)

        # Conditionally decode the image based on its extension
        img = tf.cond(is_jpg, decode_jpeg_image,
                      lambda: tf.cond(is_png, decode_png_image, skip_image))

        return img, label

    def augment_image(self, img, label):
        # Apply random flipping, brightness changes, etc.
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_brightness(img, 0.1)
        return img, label

    def build_tf_dataset(self):
        # Create dataset from file paths and labels
        dataset = tf.data.Dataset.from_tensor_slices((self.list_file_path_truth, self.list_label_truth))
        dataset = dataset.shuffle(buffer_size=self.len_dataset)  # Shuffle dataset
        dataset = dataset.map(self.load_and_preprocess_image, num_parallel_calls=2)  # Load images
        dataset = dataset.map(self.augment_image, num_parallel_calls=2)  # Data augmentation
        dataset = dataset.batch(self.batch_size)  # Batch the data
        dataset = dataset.prefetch(buffer_size=2)  # Prefetch for optimal performance
        return dataset

    def generate_minibatch(self):
        start_idx = 0
        total_batches = (self.len_dataset + self.batch_size - 1) // self.batch_size
        print("Using tf.data pipeline for minibatch generation")

        with tqdm(total=total_batches, desc="Generating Minibatches", unit="batch") as pbar:
            for batch in self.train_dataset:
                batch_img_4_truth, batch_label_truth = batch

                # Select random images for LPC loss
                batch_random_1 = []
                batch_random_2 = []
                list_random_images_path1 = random.sample(self.list_file_path_truth, k=len(batch_img_4_truth))
                list_random_images_path2 = random.sample(self.list_file_path_truth, k=len(batch_img_4_truth))

                for file_path_1 in list_random_images_path1:
                    img1 = self.load_and_preprocess_image(file_path_1, None)[0]
                    batch_random_1.append(img1)

                for file_path_2 in list_random_images_path2:
                    img2 = self.load_and_preprocess_image(file_path_2, None)[0]
                    batch_random_2.append(img2)

                batch_img_random_1 = tf.stack(batch_random_1)
                batch_img_random_2 = tf.stack(batch_random_2)
                batch_label_random = tf.zeros(len(batch_img_random_1), dtype=tf.float32)

                start_idx += self.batch_size
                pbar.update(1)
                yield batch_img_4_truth, batch_label_truth, batch_img_random_1, batch_img_random_2, batch_label_random


class MsuMsfdDataset:
    def __init__(self, batch_size, attack_dir, real_dir):  # Fixed __init__
        self.batch_size = batch_size
        self.attack_dir = attack_dir
        self.real_dir = real_dir

    def load_idx(self):
        list_file_path = []
        list_label_truth = []
        attack_dir = self.attack_dir
        real_dir = self.real_dir

        print('Loading index for attack samples...')
        for pic in os.listdir(attack_dir):
            list_file_path.append(os.path.join(attack_dir, pic))
            list_label_truth.append('attack')

        print('Loading index for real samples...')
        for pic in os.listdir(real_dir):
            list_file_path.append(os.path.join(real_dir, pic))
            list_label_truth.append('real')

        return list_file_path, list_label_truth

if __name__ == '__main__':
    msumsfd_dataset = Dataset('replayattack', 32, attack_dir=r"/content/extracted_files/dataset_small/msumsfd_gfacnn/attack",
                              real_dir=r"/content/extracted_files/dataset_small/msumsfd_gfacnn/real")

    for img_4_truth, label_truth, img_random_1, img_random_2, label_random in msumsfd_dataset.generate_minibatch():
        print(label_truth)

    # msumsfd_dataset = Dataset('replayattack', 32, attack_dir=r"/content/extracted_files/dataset_new/msumsfd_gfacnn/attack",
    #                           real_dir=r"/content/extracted_files/dataset_new/msumsfd_gfacnn/real")

    # img_4_truth, label_truth, img_random_1, img_random_2, label_random = msumsfd_dataset.generate_minibatch()

    # # Debugging: Save the images and plot them
    # for i, img in enumerate(img_4_truth[:4]):  # Save only first 4 images for debugging
    #     path = os.path.join('debug_as', f'img_{i}.jpg')
    #     cv2.imwrite(path, cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

    # h, w = 10, 10  # for raster image
    # nrows, ncols = 2, 2  # array of sub-plots
    # figsize = [6, 8]
    # fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize)

    # # plot simple raster image on each sub-plot
    # for i, axi in enumerate(ax.flat):
    #     img = img_4_truth[i]
    #     axi.imshow(img)
    #     axi.set_title(label_truth[i])

    # plt.tight_layout()
    # plt.show()

Loading index for attack samples...
Loading index for real samples...
Using tf.data pipeline for minibatch generation


Generating Minibatches:   3%|▎         | 1/36 [00:00<00:29,  1.19batch/s]

tf.Tensor([1 0 0 0 0 1 0 0 1 0 1 1 0 1 0 1 1 0 1 0 1 0 0 0 1 0 0 0 1 1 0 1], shape=(32,), dtype=int64)


Generating Minibatches:   6%|▌         | 2/36 [00:01<00:19,  1.73batch/s]

tf.Tensor([0 1 0 1 1 1 0 0 0 1 0 0 1 0 0 1 1 0 0 0 1 0 1 0 0 0 0 0 1 1 1 1], shape=(32,), dtype=int64)


...

Generating Minibatches:  94%|█████████▍| 34/36 [00:14<00:00,  2.17batch/s]

tf.Tensor([0 1 1 0 1 0 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 1 0 0 1 1 0 1 1 1 0], shape=(32,), dtype=int64)


Generating Minibatches: 100%|██████████| 36/36 [00:15<00:00,  2.88batch/s]

tf.Tensor([1 0 0 0 0 0 1 0 1 1 0 0 0 1 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 1 0 0], shape=(32,), dtype=int64)
tf.Tensor([0 1 1 1 0 0 0 0], shape=(8,), dtype=int64)


Generating Minibatches: 100%|██████████| 36/36 [00:20<00:00,  1.75batch/s]


In [ ]:
import os
import random
import yaml
import tensorflow as tf
from sklearn import preprocessing
from sklearn.utils import shuffle
from tqdm import tqdm


class Dataset:
    def __init__(self, name, batch_size, attack_dir, real_dir):  # Fixed __init__
        self.name = name
        self.batch_size = batch_size
        self.attack_dir = attack_dir
        self.real_dir = real_dir
        self.dataset = MsuMsfdDataset(batch_size, attack_dir, real_dir)

        file_path, label_truth = self.dataset.load_idx()

        # Encode the labels
        self.encoding_truth = preprocessing.LabelEncoder()
        self.encoding_truth.fit(label_truth)
        self.list_label_truth = self.encoding_truth.transform(label_truth)

        self.list_file_path_truth = file_path.copy()

        self.shuffle_dataset()
        self.len_dataset = len(self.list_file_path_truth)


        self.standard_img_size = 224

        # Build dataset using tf.data for better performance
        self.train_dataset = self.build_tf_dataset()

    def shuffle_dataset(self):
        self.list_file_path_truth, self.list_label_truth = shuffle(self.list_file_path_truth, self.list_label_truth,
                                                                   random_state=10)
    def load_and_preprocess_image(self, file_path, label):
            try:
                img = tf.io.read_file(file_path)
                img = tf.image.decode_jpeg(img, channels=3)
                img = tf.image.resize(img, (self.standard_img_size, self.standard_img_size))
                img = img / 255.0  # Normalize to [0, 1]
                return img, label
            except tf.errors.InvalidArgumentError:
                print(f"Error decoding image akash: {file_path}")
                # Handle the error, e.g., by skipping the image or using a placeholder
                return tf.zeros((self.standard_img_size, self.standard_img_size, 3)), label

    def augment_image(self, img, label):
        # Apply random flipping, brightness changes, etc.
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_brightness(img, 0.1)
        return img, label

    def build_tf_dataset(self):
        # Create dataset from file paths and labels
        dataset = tf.data.Dataset.from_tensor_slices((self.list_file_path_truth, self.list_label_truth))
        dataset = dataset.shuffle(buffer_size=self.len_dataset)  # Shuffle dataset
        dataset = dataset.map(self.load_and_preprocess_image, num_parallel_calls=2)  # Load images
        dataset = dataset.map(self.augment_image, num_parallel_calls=2)  # Data augmentation
        dataset = dataset.batch(self.batch_size)  # Batch the data
        dataset = dataset.prefetch(buffer_size=2)  # Prefetch for optimal performance
        return dataset

    def generate_minibatch(self):
        start_idx = 0
        total_batches = (self.len_dataset + self.batch_size - 1) // self.batch_size
        print("Using tf.data pipeline for minibatch generation")

        with tqdm(total=total_batches, desc="Generating Minibatches", unit="batch") as pbar:
            for batch in self.train_dataset:
                batch_img_4_truth, batch_label_truth = batch

                # Select random images for LPC loss
                batch_random_1 = []
                batch_random_2 = []
                list_random_images_path1 = random.sample(self.list_file_path_truth, k=len(batch_img_4_truth))
                list_random_images_path2 = random.sample(self.list_file_path_truth, k=len(batch_img_4_truth))

                for file_path_1 in list_random_images_path1:
                    img1 = self.load_and_preprocess_image(file_path_1, None)[0]
                    batch_random_1.append(img1)

                for file_path_2 in list_random_images_path2:
                    img2 = self.load_and_preprocess_image(file_path_2, None)[0]
                    batch_random_2.append(img2)

                batch_img_random_1 = tf.stack(batch_random_1)
                batch_img_random_2 = tf.stack(batch_random_2)
                batch_label_random = tf.zeros(len(batch_img_random_1), dtype=tf.float32)

                start_idx += self.batch_size
                pbar.update(1)
                yield batch_img_4_truth, batch_label_truth, batch_img_random_1, batch_img_random_2, batch_label_random


class MsuMsfdDataset:
    def __init__(self, batch_size, attack_dir, real_dir):  # Fixed __init__
        self.batch_size = batch_size
        self.attack_dir = attack_dir
        self.real_dir = real_dir

    def load_idx(self):
        list_file_path = []
        list_label_truth = []
        attack_dir = self.attack_dir
        real_dir = self.real_dir

        print('Loading index for attack samples...')
        for pic in os.listdir(attack_dir):
            list_file_path.append(os.path.join(attack_dir, pic))
            list_label_truth.append('attack')

        print('Loading index for real samples...')
        for pic in os.listdir(real_dir):
            list_file_path.append(os.path.join(real_dir, pic))
            list_label_truth.append('real')

        return list_file_path, list_label_truth

if __name__ == '__main__':
    msumsfd_dataset = Dataset('replayattack', 32, attack_dir=r"/content/extracted_files/dataset_new/msumsfd_gfacnn/attack",
                              real_dir=r"/content/extracted_files/dataset_new/msumsfd_gfacnn/real")

    for img_4_truth, label_truth, img_random_1, img_random_2, label_random in msumsfd_dataset.generate_minibatch():
        print(label_truth)

    # msumsfd_dataset = Dataset('replayattack', 32, attack_dir=r"/content/extracted_files/dataset_new/msumsfd_gfacnn/attack",
    #                           real_dir=r"/content/extracted_files/dataset_new/msumsfd_gfacnn/real")

    # img_4_truth, label_truth, img_random_1, img_random_2, label_random = msumsfd_dataset.generate_minibatch()

    # # Debugging: Save the images and plot them
    # for i, img in enumerate(img_4_truth[:4]):  # Save only first 4 images for debugging
    #     path = os.path.join('debug_as', f'img_{i}.jpg')
    #     cv2.imwrite(path, cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

    # h, w = 10, 10  # for raster image
    # nrows, ncols = 2, 2  # array of sub-plots
    # figsize = [6, 8]
    # fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize)

    # # plot simple raster image on each sub-plot
    # for i, axi in enumerate(ax.flat):
    #     img = img_4_truth[i]
    #     axi.imshow(img)
    #     axi.set_title(label_truth[i])

    # plt.tight_layout()
    # plt.show()

Loading index for attack samples...
Loading index for real samples...
Using tf.data pipeline for minibatch generation


Generating Minibatches:   0%|          | 1/566 [00:00<05:31,  1.70batch/s]

tf.Tensor([1 1 1 1 0 1 0 1 0 0 0 1 0 0 0 0 1 0 1 1 0 0 1 1 0 1 0 0 0 0 0 1], shape=(32,), dtype=int64)


Generating Minibatches:   0%|          | 2/566 [00:00<04:04,  2.31batch/s]

tf.Tensor([0 0 0 0 0 0 1 0 0 1 0 0 1 1 0 1 0 0 1 0 1 1 0 1 0 0 0 0 1 1 1 1], shape=(32,), dtype=int64)


...

Generating Minibatches:  29%|██▉       | 166/566 [00:58<02:07,  3.13batch/s]

tf.Tensor([1 1 0 1 1 0 0 0 1 1 1 0 0 0 0 1 1 0 0 0 0 1 0 0 1 1 0 1 1 0 1 1], shape=(32,), dtype=int64)


Generating Minibatches:  30%|██▉       | 167/566 [00:58<02:20,  2.85batch/s]

tf.Tensor([1 0 0 1 0 1 0 0 0 0 0 0 1 1 0 1 1 1 1 1 1 0 1 0 0 1 0 1 0 1 1 1], shape=(32,), dtype=int64)


InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Error in user-defined function passed to ParallelMapDatasetV2:23 transformation with iterator: Iterator::Root::Prefetch::MapAndBatch::ParallelMapV2: jpeg::Uncompress failed. Invalid JPEG data or crop window.
	 [[{{node DecodeJpeg}}]] [Op:IteratorGetNext] name: 

In [9]:
import tensorflow as tf
from tensorflow import keras
import math

# Print available physical devices
print(tf.config.experimental.list_physical_devices())

def get_model():
    # Load VGG16 with pre-trained weights and without the top layer
    vgg16 = tf.keras.applications.VGG16(
        include_top=False,
        weights="imagenet",
        input_shape=(224, 224, 3)
    )
    for layer in vgg16.layers:
        layer.trainable = False
    # Extract the output from the last layer of VGG16
    last_output = vgg16.output
    x = keras.layers.Flatten()(last_output)
    share_weight_net = keras.Model(inputs=vgg16.input, outputs=x)
    print(share_weight_net.summary())

    # Define inputs
    as_input = keras.Input(shape=(224, 224, 3), name="as_input")
    lpc_input_1 = keras.Input(shape=(224, 224, 3), name="lpc_input_1")
    lpc_input_2 = keras.Input(shape=(224, 224, 3), name="lpc_input_2")

    # Anti-spoofing branch
    as_flatten_1 = share_weight_net(as_input)
    as_fc1 = keras.layers.Dense(1024, activation="relu", name="as_fc1")(as_flatten_1)
    as_fc2 = keras.layers.Dense(1024, activation="relu", name="as_fc2")(as_fc1)
    as_output = keras.layers.Dense(2, activation='softmax', name="as_output")(as_fc2)

    # Uncomment and complete if needed
    #Local patch comparison branch
    lpc_flatten_1 = share_weight_net(lpc_input_1)
    lpc_fc_o1 = keras.layers.Dense(1024, activation="relu", name="lpc_fc1")(lpc_flatten_1)

    lpc_flatten_2 = share_weight_net(lpc_input_2)
    lpc_fc_o2 = keras.layers.Dense(1024, activation="relu", name="lpc_fc2")(lpc_flatten_2)

    lpc = keras.layers.Lambda(lambda x: tf.math.square(x[0] - x[1]), name='lpc')([lpc_fc_o1, lpc_fc_o2])

    # Define the model
    model = keras.Model(
        inputs=[as_input,lpc_input_1,lpc_input_2],
        outputs=[as_output,lpc]
    )

    # Compile the model with appropriate losses and metrics
    model.compile(
        optimizer='adam',
        loss={
            "as_output": keras.losses.SparseCategoricalCrossentropy(),
            "lpc": tpc_loss},
        loss_weights={
            "as_output": 1.0,
            "lpc": 2.5*math.exp(-5)
        },
        metrics={"as_output": 'accuracy'}
    )

    return model

def tpc_loss(y_true, y_pred):
    return tf.reduce_sum(y_pred, axis=1, keepdims=True)

if __name__ == '__main__':
    model = get_model()
    model.summary()
    #Uncomment to plot model architecture
    tf.keras.utils.plot_model(
        model,
        to_file='model1.png',
        show_shapes=True,
        show_dtype=False,
        show_layer_names=True,
        rankdir='TB',
        expand_nested=False,
        dpi=200,
        show_layer_activations=True,
    )

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 25088)               │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 0 (0.00 B)

...

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ as_input (InputLayer)     │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ functional_4 (Functional) │ (None, 25088)          │     14,714,688 │ as_input[0][0],        │
│                           │                        │                │ lpc_input_1[0][0],     │
│                           │                        │                │ lpc_input_2[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lpc_input_1 (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lpc_input_2 (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ as_fc1 (Dense)            │ (None, 1024)           │     25,691,136 │ functional_4[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ as_fc2 (Dense)            │ (None, 1024)           │      1,049,600 │ as_fc1[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lpc_fc1 (Dense)           │ (None, 1024)           │     25,691,136 │ functional_4[1][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lpc_fc2 (Dense)           │ (None, 1024)           │     25,691,136 │ functional_4[2][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ as_output (Dense)         │ (None, 2)              │          2,050 │ as_fc2[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lpc (Lambda)              │ (None, 1024)           │              0 │ lpc_fc1[0][0],         │
│                           │                        │                │ lpc_fc2[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 92,839,746 (354.16 MB)

 Trainable params: 78,125,058 (298.02 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [ ]:
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from tqdm import tqdm

# Configure GPU memory growth (if needed)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

if __name__ == '__main__':
    # with open("E:/akash singh/GFA-CNN-new/GFA-CNN-master/GFA-CNN-master/config.yaml", "r") as ymlfile:
    #     cfg = yaml.safe_load(ymlfile)  # Use safe_load for security

    epoch = 2 #cfg['training']['epoch']
    batch = 32# cfg['training']['batch']

    attack_dir = r"/content/extracted_files/dataset_small/msumsfd_gfacnn/attack"
    real_dir = r"/content/extracted_files/dataset_small/msumsfd_gfacnn/real"
    dataset = Dataset('MSU-MSFD', batch_size=batch, attack_dir=attack_dir, real_dir=real_dir)
    GFA_CNN = get_model()

    checkpoint_filepath = './checkpoints/chkpt.weights.h5'
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_weights_only=True,
        monitor='loss',
        mode='min',
        save_best_only=True)

    print('Preprocessing data ...')

    print('Training ...')
    j = 1
    for e in tqdm(range(epoch), desc='Epochs'):
        print(f"[epoch number: {j}]")
        dataset.shuffle_dataset()
        i = 1
        for batch_img_4_truth, batch_label_truth, batch_img_random_1, batch_img_random_2, batch_label_random in tqdm(
                dataset.generate_minibatch(), desc='Batches', total=len(dataset.list_file_path_truth) // batch):
            GFA_CNN.fit(
                {"as_input": batch_img_4_truth, "lpc_input_1": batch_img_random_1, "lpc_input_2": batch_img_random_2},
                {"as_output": batch_label_truth, 'lpc': batch_label_random},
                epochs=4,
                batch_size=16,
                callbacks=[model_checkpoint_callback],
                verbose=1
            )
            print(f"counting iterations {i}")
            i += 1
        j += 1

    # print("Saving the model...")
    # model_save_path = '/content/model_saved'
    # GFA_CNN.save(model_save_path)
    # print(f"Model saved to {model_save_path}")

    # Load the testing dataset
    attack_dir_test = r"/content/extracted_files/dataset_small/msumsfd_gfacnn_test/attack"
    real_dir_test = r"/content/extracted_files/dataset_small/msumsfd_gfacnn_test/real"
    test_dataset = Dataset('MSU-MSFD', batch_size=batch, attack_dir=attack_dir_test, real_dir=real_dir_test)

    y_true = []
    y_pred = []

    for batch_img_4_truth, batch_label_truth, batch_img_random_1, batch_img_random_2 in test_dataset.generate_minibatch():
        predictions = GFA_CNN.predict({
            "as_input": batch_img_4_truth,
            "lpc_input_1": batch_img_random_1,
            "lpc_input_2": batch_img_random_2
        })

        y_true.extend(batch_label_truth)
        y_pred.extend(predictions[0])  # Adjust according to your output names

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_pred = y_pred[:, 0]  # Assuming first column represents positive class probability

    def calculate_eer(y_true, y_pred):
        fpr, tpr, thresholds = roc_curve(y_true, y_pred)
        fnr = 1 - tpr
        eer_threshold = thresholds[np.nanargmin(np.abs(fnr - fpr))]
        eer = fpr[np.nanargmin(np.abs(fnr - fpr))]
        return eer

    def calculate_hter(eer, y_true, y_pred):
        fpr, tpr, _ = roc_curve(y_true, y_pred)
        fnr = 1 - tpr
        threshold = np.interp(eer, fpr, tpr)
        hter = (fpr[np.argmin(np.abs(threshold - tpr))] + fnr[np.argmin(np.abs(threshold - fnr))]) / 2
        return hter

    eer = calculate_eer(y_true, y_pred)
    print(f"Equal Error Rate (EER): {eer:.4f}")

    hter = calculate_hter(eer, y_true, y_pred)
    print(f"Half Total Error Rate (HTER): {hter:.4f}")

    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    plt.figure()
    plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (area = %0.2f)' % auc(fpr, tpr))
    plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate (FPR)')
    plt.ylabel('True Positive Rate (TPR)')
    plt.title('Receiver Operating Characteristic (ROC)')
    plt.legend(loc='lower right')
    plt.grid(True)
    plt.show()

Loading index for attack samples...
Loading index for real samples...


Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 25088)               │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 0 (0.00 B)

...

[epoch number: 1]



Batches:   0%|          | 0/35 [00:00<?, ?it/s]

Using tf.data pipeline for minibatch generation




Generating Minibatches:   0%|          | 0/36 [00:00<?, ?batch/s]

Generating Minibatches:   3%|▎         | 1/36 [00:00<00:24,  1.44batch/s]

Epoch 1/4
